<a href="https://colab.research.google.com/github/DeveloperVivek9/Calorie-Tracker/blob/main/Deployment/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"
!pip install detecto --quiet

     |████████████████████████████████| 123 kB 12.7 MB/s 
albumentations==1.3.0 is successfully installed


In [ ]:
import albumentations as A
import cv2
from detecto.utils import xml_to_csv
#from google.colab import drive
import numpy as np
import os
import pandas as pd
from PIL import Image
import re
import shutil

In [ ]:
#PROBLEMS SOLVED
#Transformed images of Images having different extensions will be in jpg which may cause problems. Conserve extension info
#Logic not written for images with multiple bounding boxes. 
#       Transformed images getting saved again and again if it faces multiple bounding boxes.
#       Image id is not image id here but rather it's the number of additions onto csv
#If already transformed, it will do transformation on that file which will cause problems
#Name of image changed to function name so that we get accurate information from name itself 
#else too many if statements needed which decrease readability
#4. Sometimes transformed bndbox empty. Is it because random rotation or something? 
#       We tested on the same image multiple times. Rotate alone giving no bounding box the second time onwards. [?]
#Solution -> Just check length of what is returned. But why is this  happening?
#1. Transform applied to each record in labels.csv which is optimal but transformed image created for each bndbox which is not.
#       Create transformed images for each unique image
#       Add transformed bounding boxes to csv
# Solution -> don't save image if image already transformed. [dictionary where each key value -> transform name, value -> list of images done]
#2. Why does compose require list(tuple())?? -> It should be because it is expecting multiple bndboxes 
#       since we are iterating for each bndbox, list not required, since it is expecting one, we should follow.

In [ ]:
#I don't want augmentations to be done along data entry function
#Reason -> we won't be able to change augmentation functions or add more augmentations 
#Here logic written so that transformed images won't be transformed again. 

In [ ]:
#ISSUES
#3. Fix visualization function. Bounding box wrong. 
#When augmentation function gets changed, we should remove those transformed images and add them again. [logic for that should be written]
#Tranform list changes should reflect in csv file too. Removing a transformation should delete those transformed images from csv file and delete them.

In [ ]:
def visualize_bbox(img, bbox, class_name, BOX_COLOR,TEXT_COLOR, thickness=2):
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=BOX_COLOR, thickness=thickness)
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img

In [ ]:
def visualize(image, bndbox,class_name,BOX_COLOR=(255, 0, 0),TEXT_COLOR=(255, 255, 255)):
    img = image.copy()
    img = visualize_bbox(img, bndbox,class_name,BOX_COLOR,TEXT_COLOR)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [ ]:
#Augmentation functions used on the images. Note the name of the function which is what should be in total_transform_list.
def Rotate():
  return A.Compose([A.ShiftScaleRotate(always_apply=True,p=0.5)],bbox_params=A.BboxParams(format='pascal_voc'))

def HFlip():
  return A.Compose([A.HorizontalFlip(always_apply=True,p=0.5)],bbox_params=A.BboxParams(format='pascal_voc'))

def Brighten():
  return A.Compose([A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, brightness_by_max=True, always_apply=True, p=0.5)],bbox_params=A.BboxParams(format='pascal_voc'))

In [ ]:
#Data augmentation code 
def data_augment(img_path,class_name,bndbox_with_class,transform_name):
  """
  Applies transform_name on the image. 
  Returns transformed image and transformed bounding boxes. 
  """
  image = cv2.imread(img_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  #Calling relevant function
  transform = globals()[transform_name] 
  #Calling the function
  transform=transform()
  #Applying transformation
  transformed = transform(image=image, bboxes=[bndbox_with_class], class_labels=[class_name])
  transformed_bboxes = transformed['bboxes']
  transformed_image = transformed['image']
  return transformed_image,transformed_bboxes #Return the transformed images,bboxes and the name

In [ ]:
def Augmentation_function(base_path,Image_Metadata_csv,total_transform_list=['Brighten','HFlip','Rotate']):

  """
  Augments images listed in Image Metadata.csv where base_path is the path of the csv file. 
  total_transform_list is the list of transformations we need to do on the images.
  """

  #If image transformed, new name added to list. We use this to get image id.
  transformed_images_list=list()

  #Transform dict is to get transformations done on a particular image.
  #key -> image name without extension.
  #value -> list of transformations done on that image.
  #We are assuming no transformation has been done on transformed images. [Better to create augmentation pipeline in the respective function instead of applying again]
  labels=pd.read_csv(base_path+Image_Metadata_csv)
  Transform_dict=dict()
  for x in list(labels['Image Name']):
    try:
      Unique_id=x.replace(re.search(r"[ A-Za-z]+\.[A-Za-z]+", x).group(),"")
      transform_function=re.search(r"[A-Za-z]+\.[A-Za-z]+", x).group().split('.', 1)[0] 
      Transform_dict[Unique_id].append(transform_function)
    except KeyError:
      Transform_dict[Unique_id]=list()
      Unique_id=x.replace(re.search(r"[ A-Za-z]+\.[A-Za-z]+", x).group(),"")
      transform_function=re.search(r"[A-Za-z]+\.[A-Za-z]+", x).group().split('.', 1)[0] 
      Transform_dict[Unique_id].append(transform_function)
    except AttributeError:
      continue
  #print(Transform_dict)
  #Start and end indices to get how many records added.   
  start_index=labels['Image ID'].max()+1
  end_index=start_index
  #To traverse each bounding box in the dataframe
  for index, row in labels.iterrows(): 
    image_name=row['Image Name']
    class_name = row['Class']
    image_path=row['Image Path']
    bndbox_with_class = [row['xmin'],row['ymin'],row['xmax'],row['ymax'],row['Class']] 
    print("In ",row)
    #Avoiding augmented images.
    try:
      #Regex to check if string after image id -> which means augmented image
      transform_name=re.search(r"[A-Za-z]+\.[A-Za-z]+", image_name).group().split('.', 1)[0]   
      """
      if(transform_name not in total_transform_list):
        os.remove(labels.loc[index,'Image Path'])
        labels.drop(axis=0, index=index,inplace=True)
      """
      continue 
    except AttributeError:
      pass

    try:
      transform_list=[x for x in total_transform_list if x not in Transform_dict[os.path.splitext(image_name)[0]]]
    except KeyError:
      transform_list=total_transform_list

    #Looping through transforms not done on that image :
    #i) save transformed image if not saved 
    #ii) write transformed bounding box onto csv
    for transform_name in transform_list:
      print("Image name : ",image_name,"Transform Name : ",transform_name)
      transformed_image,transformed_bndbox = data_augment(image_path+image_name,class_name,bndbox_with_class,transform_name)
      #print(transformed_bndbox,bndbox_with_class)
      if(len(transformed_bndbox)<1):
        #print(len(transformed_bndbox))
        continue
      name_of_file = os.path.splitext(image_name)[0] + " "+transform_name+os.path.splitext(image_name)[1]
      if(name_of_file not in transformed_images_list):
        transformed_images_list.append(name_of_file)       
        im = Image.fromarray(transformed_image)
        im.save(image_path+name_of_file)
      image_id=start_index+transformed_images_list.index(name_of_file)
      width = im.width
      height = im.height
      xmin,ymin,xmax,ymax= [int(x) for x in transformed_bndbox[0][0:4]]
      labels.loc[len(labels.index)] = [len(labels.index),image_id,image_path,name_of_file,height,width,class_name,xmin,ymin,xmax,ymax,None]
      end_index+=1
  labels.to_csv(base_path+Image_Metadata_csv,encoding='utf-8',index=False)
  return start_index,end_index

In [ ]:
"""
drive.mount('/content/drive', force_remount=True)
"""

Mounted at /content/drive


In [ ]:
"""
base_path='/content/drive/MyDrive/Experiment/Unannotated/'
disparity_dir='/content/drive/MyDrive/Experiment/Disparity/'
destination_dir='/content/drive/MyDrive/Experiment/Annotated/'
Image_Metadata_csv='/content/drive/MyDrive/Experiment/Image Metadata.csv'
Augmentation_function(destination_dir,Image_Metadata_csv)
"""

In  Bndbox ID                                                0
Image ID                                                 0
Image Path    /content/drive/MyDrive/Experiment/Annotated/
Image Name                             Aamla Murabba 0.jpg
Height                                                 540
Width                                                  720
Class                                        Aamla Murabba
xmin                                                   206
ymin                                                     6
xmax                                                   470
ymax                                                   233
Batch                                                  NaN
Name: 0, dtype: object
Image name :  Aamla Murabba 0.jpg Transform Name :  Brighten
Image name :  Aamla Murabba 0.jpg Transform Name :  HFlip
Image name :  Aamla Murabba 0.jpg Transform Name :  Rotate
In  Bndbox ID                                                1
Image ID                

(2, 11)

In [ ]:
"""
labels=pd.read_csv(Image_Metadata_csv)
"""

In [ ]:
"""
for index, row in labels.iterrows(): 
    image_name=row['Image Name']
    class_name = row['Class']
    image_path=row['Image Path']+image_name
    print(image_name)
"""

Aaloo Chips 0.jpg
Aaloo Chips 0.jpg
Aamla Murabba 1.jpg
Aaloo Chips 0 Brighten.jpg
Aaloo Chips 0 HFlip.jpg
Aaloo Chips 0 Rotate.jpg
Aamla Murabba 1 Brighten.jpg
Aamla Murabba 1 HFlip.jpg
Aamla Murabba 1 Rotate.jpg


In [ ]:
"""
#Directory containing images  and xml files
base_path=""
#csv file generated in Data Preprocessing Pipeline
labels=pd.read_csv("Image Metadata.csv")
"""